In [ ]:
# import relevant libraries
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import os
import pandas as pd

In [2]:
# youtube api key
api_key = os.getenv("API_KEY")

# build api client
api_name = "youtube"
api_version = "v3"
youtube = build(api_name, api_version, developerKey=api_key)

In [ ]:
# search for the first 100 video results for each query
queries = ["animal liberation", "vegan speech", "vegan outreach", "vegan lifestyle", "joey carbstrong", "earthling ed",
            "david ramms", "vegan gains", "wayne hsiung", "mic the vegan"]
video_ids = []

for query in queries:
    next_page_token = None

    for _ in range(2):
        request = youtube.search().list(
            part="id",
            type="video",
            q=query,
            maxResults=50,
            relevanceLanguage="en",
            order="relevance",
            pageToken=next_page_token
        )
        try:
            response = request.execute()
            video_ids += [item["id"].get("videoId") for item in response["items"]]
            next_page_token = response.get("nextPageToken")
        except HttpError as e:
            print(f"Error response status code: {e.status_code}, reason: {e.error_details}")
            raise
        if not next_page_token:
            break
video_ids

In [15]:
# de-duplicate video list and remove None values
unique_vid_ids = list(set([vid for vid in video_ids if vid is not None]))

# scrape data on unique videos
video_data = []

for i in range(0, len(unique_vid_ids), 50):
    request = youtube.videos().list(
        part="snippet, contentDetails, statistics",
        id=",".join(unique_vid_ids[i:i+50]),
        maxResults=50
    )
    try:
        response = request.execute()

        for item in response["items"]:
            snippet = item["snippet"]
            content = item["contentDetails"]
            stats = item.get("statistics", {})

            data = {
                "video_id": item["id"],
                "title": snippet.get("title"),
                "description": snippet.get("description"),
                "published_at": snippet.get("publishedAt"),
                "channel_title": snippet.get("channelTitle"),
                "category_id": snippet.get("categoryId"),
                "tags": snippet.get("tags", []),
                "duration": content.get("duration"),
                "definition": content.get("definition"),
                "caption": content.get("caption"),
                "view_count": int(stats.get("viewCount", 0)),
                "like_count": int(stats.get("likeCount", 0)),
                "comment_count": int(stats.get("commentCount", 0))
            }
            video_data.append(data)
    except HttpError as e:
        print(f"Error response status code: {e.status_code}, reason: {e.error_details}")
        raise

In [16]:
video_df = pd.DataFrame(video_data)
video_df.head()

,video_id,title,description,published_at,channel_title,category_id,tags,duration,definition,caption,view_count,like_count,comment_count
0,OKOeUqKvavw,The Speech That Turned Me Into A Vegan - Vegan...,"Okon Bros chat with Vegan Gains, AKA Richard B...",2022-08-30T00:59:43Z,Okon Bros,24,"[richard burgess, richard burgess vegan gains,...",PT33S,hd,false,19244,295,94
1,OLpT9Dd9AqE,(Pt.2) Animal Liberation: Peter Singer Speaks ...,"Author. Lecturer. Philosopher. In 1975, Peter ...",2012-08-10T12:41:55Z,Alliance for Animals and the Environment,29,"[peter singer, animal rights, madison, wiscons...",PT25M22S,sd,false,306,0,0
2,AUdMTtaIWsY,BOLD NATIVE : a film about animal liberation,Support the creators of this film with a donat...,2017-03-26T14:43:21Z,VeganRevolution,29,"[Vegan, Revolution, Veganism, Bold Native, Ful...",PT1H44M47S,hd,false,4596,319,53
3,MT4I8UijYB4,VEGAN GIVES SECURITY GUARD THE SHOCK OF HIS LI...,He certainly wasn't prepared for this. \n\n*Wa...,2018-07-25T21:57:19Z,Joey Carbstrong,24,[],PT11M55S,hd,false,37523,2507,569
4,8WSrRV5ovPE,EVERY Vegan Needs to Hear This! Are You Advoca...,Are you advocating cruelty in your vegan activ...,2022-03-16T12:51:21Z,BiteSizeVegan.org,29,"[bite size vegan, veganism, vegan, Emily Moran...",PT35M39S,hd,true,6662,703,145


In [ ]:
# save to csv
video_df.to_csv("data/animal_rights_videos_unfiltered", index=False)